In [1]:
import os
os.environ['MPLCONFIGDIR'] = "/home/mventura/latent_ode_expts"
import sys
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot
import matplotlib.pyplot as plt

import time
import datetime
import argparse
import numpy as np
import pandas as pd
from random import SystemRandom
from sklearn import model_selection

import torch
import torch.nn as nn
from torch.nn.functional import relu
import torch.optim as optim

import lib.utils as utils
from lib.plotting import *

from lib.rnn_baselines import *
from lib.ode_rnn import *
from lib.create_latent_ode_model import create_LatentODE_model
from lib.parse_datasets import parse_datasets
from lib.ode_func import ODEFunc, ODEFunc_w_Poisson
from lib.diffeq_solver import DiffeqSolver
from mujoco_physics import HopperPhysics

from lib.utils import compute_loss_all_batches
%load_ext autoreload 
%autoreload 2

Matplotlib created a temporary config/cache directory at /tmp/matplotlib-gxc55qvv because the default path (/home/mventura/latent_ode_expts) is not a writable directory; it is highly recommended to set the MPLCONFIGDIR environment variable to a writable directory, in particular to speed up the import of Matplotlib and to better support multiprocessing.


# LatentODE(10)

In [2]:
# !python3 run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic --latent-ode --noise-weight 0 --viz

In [3]:
# !python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 10 --dataset periodic --latent-ode --noise-weight 0 --viz

latent dim = 10, missing = 0, noise = 0

In [4]:
!python3 run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 1 -l 10 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 74924
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 2588.466797 | Likelihood -2596.165283 | KL fp 2.4647 | FP STD 0.5095|
KL coef: 0.0
Train loss (one batch): 2876.571533203125
Train CE loss (one batch): 0.0
Test MSE: 0.5200
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74924
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 2350.493652 | Likelihood -2356.909668 | KL fp 5.2577 | FP STD 0.5017|
KL coef: 0.0
Train loss (one batch): 2389.11474609375
Train CE loss (one batch): 0.0
Test MSE: 0.4721
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74924
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 2306.531738 | Likelihood -2308.807861 | KL fp 6.1342 | FP STD 0.3257|
KL coef: 0.0
Train loss (one 

plotting....
Computing loss... 0
Experiment 74924
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 752.811768 | Likelihood -763.432800 | KL fp 4.1219 | FP STD 0.0847|
KL coef: 0.15705680661607324
Train loss (one batch): 641.3211059570312
Train CE loss (one batch): 0.0
Test MSE: 0.1534
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74924
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 647.102783 | Likelihood -655.130615 | KL fp 4.5271 | FP STD 0.0632|
KL coef: 0.1654862385499125
Train loss (one batch): 614.7162475585938
Train CE loss (one batch): 0.0
Test MSE: 0.1318
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 74924
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 673.881714 | Likelihood -675.959045 | KL fp 4.8999 | FP STD 0.0412|
KL coef: 0.17383137616441335
Train loss (one batch): 757.6233520507812
Train CE loss (one batch): 0.0
Test MSE: 0.1359
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss..

latent dim = 10, missing = 50, noise = 0

In [5]:
!python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 10 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 0.5 -l 10 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 44991
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 2411.389893 | Likelihood -2420.495361 | KL fp 2.8903 | FP STD 0.6186|
KL coef: 0.0
Train loss (one batch): 2568.56591796875
Train CE loss (one batch): 0.0
Test MSE: 0.4848
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 44991
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 2312.475586 | Likelihood -2314.508545 | KL fp 5.0289 | FP STD 0.2426|
KL coef: 0.0
Train loss (one batch): 2333.6455078125
Train CE loss (one batch): 0.0
Test MSE: 0.4636
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 44991
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 2295.954590 | Likelihood -2300.543213 | KL fp 4.9047 | FP STD 0.3202|
KL coef: 0.0
Train loss (one 

plotting....
Computing loss... 0
Experiment 44991
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 899.765869 | Likelihood -928.081543 | KL fp 4.1156 | FP STD 0.1391|
KL coef: 0.15705680661607324
Train loss (one batch): 1505.193115234375
Train CE loss (one batch): 0.0
Test MSE: 0.1864
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 44991
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 936.660034 | Likelihood -961.515076 | KL fp 4.1811 | FP STD 0.1328|
KL coef: 0.1654862385499125
Train loss (one batch): 729.1923217773438
Train CE loss (one batch): 0.0
Test MSE: 0.1930
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 44991
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 1209.026123 | Likelihood -1213.370605 | KL fp 4.5696 | FP STD 0.0852|
KL coef: 0.17383137616441335
Train loss (one batch): 743.7263793945312
Train CE loss (one batch): 0.0
Test MSE: 0.2434
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss

# LatentODE(5)

In [6]:
# !python3 run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic --latent-ode --noise-weight 0 --viz

In [7]:
# !python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 5 --dataset periodic --latent-ode --noise-weight 0 --viz

latent dim = 5, missing = 0, noise = 0

In [8]:
!python3 run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 1 -l 5 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 41559
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 2437.495361 | Likelihood -2439.148682 | KL fp 3.4149 | FP STD 0.4424|
KL coef: 0.0
Train loss (one batch): 2486.174560546875
Train CE loss (one batch): 0.0
Test MSE: 0.4886
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 41559
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 2372.867432 | Likelihood -2375.984375 | KL fp 5.2688 | FP STD 0.2810|
KL coef: 0.0
Train loss (one batch): 2391.18896484375
Train CE loss (one batch): 0.0
Test MSE: 0.4759
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 41559
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 2339.658447 | Likelihood -2341.208984 | KL fp 5.5379 | FP STD 0.2389|
KL coef: 0.0
Train loss (one b

plotting....
Computing loss... 0
Experiment 41559
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 1945.055542 | Likelihood -1944.931763 | KL fp 11.5672 | FP STD 0.1653|
KL coef: 0.15705680661607324
Train loss (one batch): 1988.33544921875
Train CE loss (one batch): 0.0
Test MSE: 0.3897
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 41559
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 1858.098145 | Likelihood -1858.088379 | KL fp 11.4152 | FP STD 0.1313|
KL coef: 0.1654862385499125
Train loss (one batch): 1885.6336669921875
Train CE loss (one batch): 0.0
Test MSE: 0.3724
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 41559
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 1766.602051 | Likelihood -1766.610840 | KL fp 12.2448 | FP STD 0.0710|
KL coef: 0.17383137616441335
Train loss (one batch): 1747.2340087890625
Train CE loss (one batch): 0.0
Test MSE: 0.3541
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Comput

latent dim = 5, missing = 50, noise = 0

In [9]:
!python3 run_models.py --niters 50 -n 1000 -s 0.5 -l 5 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz

Sampling dataset of 1000 training examples
/home/phd/mventura/latent_ode_expt/run_models.py
run_models.py --niters 50 -n 1000 -s 0.5 -l 5 --dataset periodic_d1 --latent-ode --noise-weight 0 --viz
Computing loss... 0
Experiment 62661
Epoch 0001 [Test seq (cond on sampled tp)] | Loss 2504.298340 | Likelihood -2509.859619 | KL fp 3.9696 | FP STD 0.3292|
KL coef: 0.0
Train loss (one batch): 2419.79248046875
Train CE loss (one batch): 0.0
Test MSE: 0.5027
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 62661
Epoch 0002 [Test seq (cond on sampled tp)] | Loss 2369.623779 | Likelihood -2372.223389 | KL fp 5.1546 | FP STD 0.2587|
KL coef: 0.0
Train loss (one batch): 2394.1298828125
Train CE loss (one batch): 0.0
Test MSE: 0.4752
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 62661
Epoch 0003 [Test seq (cond on sampled tp)] | Loss 2338.631104 | Likelihood -2340.405518 | KL fp 5.7051 | FP STD 0.2273|
KL coef: 0.0
Train loss (one b

plotting....
Computing loss... 0
Experiment 62661
Epoch 0027 [Test seq (cond on sampled tp)] | Loss 2058.421631 | Likelihood -2057.792236 | KL fp 11.0525 | FP STD 0.0647|
KL coef: 0.15705680661607324
Train loss (one batch): 2073.140625
Train CE loss (one batch): 0.0
Test MSE: 0.4123
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 62661
Epoch 0028 [Test seq (cond on sampled tp)] | Loss 2047.772827 | Likelihood -2047.165527 | KL fp 10.8046 | FP STD 0.0910|
KL coef: 0.1654862385499125
Train loss (one batch): 2058.06201171875
Train CE loss (one batch): 0.0
Test MSE: 0.4102
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing loss... 0
Experiment 62661
Epoch 0029 [Test seq (cond on sampled tp)] | Loss 2026.183838 | Likelihood -2025.524048 | KL fp 11.1447 | FP STD 0.0837|
KL coef: 0.17383137616441335
Train loss (one batch): 2040.8668212890625
Train CE loss (one batch): 0.0
Test MSE: 0.4058
Poisson likelihood: 0.0
CE loss: 0.0
plotting....
Computing los